# Following the Options

Strategy: Buy equity when call open interest goes up (by some amount) after earnings release, and short equity when put open interest goes up after earnings release.

In [4]:
import sys
sys.path.append('../../utils/')

from query import query_dataframe, get_connection
from trading_days import TradingDay

date_format = '%Y-%m-%d'

In [5]:
import pandas as pd

earnings_dates = pd.read_csv('all_earnings.csv', index_col=0)
earnings_dates['NormDate'] = pd.to_datetime(
    earnings_dates['NormDate'],
    #format="%m-%d-%Y",
    format='%Y-%m-%d',
    errors='coerce'
)

earnings_dates = earnings_dates[(earnings_dates['Date'] > '2011-01-01') &
                                (earnings_dates['Date'] < '2014-01-01')]
earnings_dates['BeforeOpen'] = earnings_dates['BeforeOpen'].astype(bool)

In [9]:
import sqlalchemy
earnings_dates.to_sql(
    'bcs2149_earnings', get_connection(),
    dtype={
        'Date': sqlalchemy.types.DATE,
        'NormDate': sqlalchemy.types.DATE
    }
)

import pymssql
import time

query_file = open('OpenInterestShift.sql', 'r')
base_query = query_file.read()

def build_condition(ticker, earnings_date, prior_date, close_date):
    return ("(sn.Ticker = {ticker} AND op.Date = '{earnings_date}' AND"
            " op_before.Date = '{prior_date}' AND sp_after.Date = '{close_date})'"
            .format(
            ticker=ticker,
            earnings_date=earnings_date.strftime(date_format),
            prior_date=prior_date.strftime(date_format),
            close_date=close_date.strftime(date_format)
        ))

def open_interest_data(row, open_seek, equity_hold):
    ticker = row['Ticker']
    earnings_date = row['NormDate']
    prior_date = earnings_date - TradingDay(open_seek)
    close_date = earnings_date + TradingDay(equity_hold)
    
    return build_condition(ticker, earnings_date, prior_date, close_date)

datasets = [
    #(1, 20),
    (1, 10),
    (1, 15),
    (1, 30),
    (2, 20),
    (2, 10),
    (2, 15),
    (2, 30)
]

for open_seek, equity_hold in datasets:
    print("Pulling data for open_seek={} and equity_hold={}".format(
            open_seek, equity_hold
        ))
    print("Building query...")
    query_conditions = [open_interest_data(row, open_seek, equity_hold)
                        for i, row in earnings_dates.iterrows()]
    full_query = base_query.format(conditions=' OR '.join(query_conditions))
    print("Executing query...")
    trade_data = query_dataframe(full_query)
    trade_data.to_csv('open_interest_{}_{}.csv'.format(
            open_seek, equity_hold
        ))